# Project 4: Predicting the Stock Market Trends with RNN and LTSM

## LTSM

### Importing Libraries

In [7]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

### Importing Dataset

In [8]:
fb_complete_data = pd.read_csv('data/fb_train.csv')
fb_complete_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2015-01-02,78.580002,78.930000,77.699997,78.449997,78.366852,18177500
1,2015-01-05,77.980003,79.250000,76.860001,77.190002,77.108192,26452200
2,2015-01-06,77.230003,77.589996,75.360001,76.150002,76.069290,27399300
3,2015-01-07,76.760002,77.360001,75.820000,76.150002,76.069290,22045300
4,2015-01-08,76.739998,78.230003,76.080002,78.180000,78.097137,23961000


In [9]:
fb_trainging_processed = fb_complete_data[["Open"]].values

In [11]:
scalar = MinMaxScaler(feature_range=(0,1))

fb_training_scaled = scalar.fit_transform(fb_trainging_processed)

In [12]:
len(fb_training_scaled)

1257

In [13]:
fb_training_features = []
fb_training_labels = []

for i in range(60,len(fb_training_scaled)):
    fb_training_features.append(fb_training_scaled[i-60:i,0])
    fb_training_labels.append(fb_training_scaled[i,0])

In [14]:
X_train = np.array(fb_training_features)
y_train = np.array(fb_training_labels)

In [15]:
print(X_train.shape)
print(y_train.shape)

(1197, 60)
(1197,)


In [16]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))